In [25]:
from metar import Metar
import re
import datetime
import warnings
import logging

In [22]:
# Test Function
def decode(message):
    obs = Metar.Metar(message,strict=False)
    print(obs.string())
# Using Metar Function
def ConvertToCSV(message):
    obs = Metar.Metar(message,strict=False)
    line = []
    # Station ID
    line.append(obs.station_id)
    # Time
    line.append(obs.time.ctime())
    # temperature
    line.append(obs.temp.string("C"))
    # dewpt
    line.append(obs.dewpt.string("C"))
    # wind
    line.append(obs.wind())
    # visibility
    line.append(obs.visibility())
    # pressure
    line.append(obs.press.string("mb"))
    # sky
    line.append(obs.sky_conditions(", "))
    # Remark
    line.append("- " + " ".join(obs._unparsed_remarks))
    
    return '\t'.join(line)

In [47]:
# Grab Raw Data 
# regular expressions to decode various groups of the METAR code
MISSING_RE = re.compile(r"^[M/]+$")
TYPE_RE = re.compile(r"^(?P<type>METAR|SPECI)\s+")
COR_RE = re.compile(r"^(?P<cor>COR)\s+")
STATION_RE = re.compile(r"^(?P<station>[A-Z][A-Z0-9]{3})\s+")
TIME_RE = re.compile(
    r"""^(?P<day>\d\d)
        (?P<hour>\d\d)
        (?P<min>\d\d)Z?\s+""",
    re.VERBOSE,
)
MODIFIER_RE = re.compile(r"^(?P<mod>AUTO|FINO|NIL|TEST|CORR?|RTD|CC[A-G])\s+")
WIND_RE = re.compile(
    r"""^(?P<dir>[\dO]{3}|[0O]|///|MMM|VRB)
        (?P<speed>P?[\dO]{2,3}|[/M]{2,3})
        (G(?P<gust>P?(\d{1,3}|[/M]{1,3})))?
        (?P<units>KTS?|LT|K|T|KMH|MPS)?
        (\s+(?P<varfrom>\d\d\d)V
        (?P<varto>\d\d\d))?\s+""",
    re.VERBOSE,
)
VISIBILITY_RE = re.compile(
    r"""^(?P<vis>(?P<dist>(M|P)?\d\d\d\d|////)
        (?P<dir>[NSEW][EW]? | NDV)? |
        (?P<distu>(M|P)?(\d+|\d\d?/\d\d?|\d+\s+\d/\d))
        (?P<units>SM|KM|M|U) |
        CAVOK )\s+""",
    re.VERBOSE,
)
RUNWAY_RE = re.compile(
    r"""^(RVRNO |
        R(?P<name>\d\d(RR?|LL?|C)?)/
        (?P<low>(M|P)?\d\d\d\d)
        (V(?P<high>(M|P)?\d\d\d\d))?
        (?P<unit>FT)?[/NDU]*)\s+""",
    re.VERBOSE,
)
WEATHER_RE = re.compile(
    r"""^(?P<int>(-|\+|VC)*)
        (?P<desc>(MI|PR|BC|DR|BL|SH|TS|FZ)+)?
        (?P<prec>(DZ|RA|SN|SG|IC|PL|GR|GS|UP|/)*)
        (?P<obsc>BR|FG|FU|VA|DU|SA|HZ|PY)?
        (?P<other>PO|SQ|FC|SS|DS|NSW|/+)?
        (?P<int2>[-+])?\s+""",
    re.VERBOSE,
)
SKY_RE = re.compile(
    r"""^(?P<cover>VV|CLR|SKC|SCK|NSC|NCD|BKN|SCT|FEW|[O0]VC|///)
        (?P<height>[\dO]{2,4}|///)?
        (?P<cloud>([A-Z][A-Z]+|///))?\s+""",
    re.VERBOSE,
)
TEMP_RE = re.compile(
    r"""^(?P<temp>(M|-)?\d+|//|XX|MM)/
        (?P<dewpt>(M|-)?\d+|//|XX|MM)?\s+""",
    re.VERBOSE,
)
PRESS_RE = re.compile(
    r"""^(?P<unit>A|Q|QNH)?
        (?P<press>[\dO]{3,4}|////)
        (?P<unit2>INS)?\s+""",
    re.VERBOSE,
)
RECENT_RE = re.compile(
    r"""^RE(?P<desc>MI|PR|BC|DR|BL|SH|TS|FZ)?
        (?P<prec>(DZ|RA|SN|SG|IC|PL|GR|GS|UP)*)?
        (?P<obsc>BR|FG|FU|VA|DU|SA|HZ|PY)?
        (?P<other>PO|SQ|FC|SS|DS)?\s+""",
    re.VERBOSE,
)
WINDSHEAR_RE = re.compile(r"^(WS\s+)?(ALL\s+RWY|RWY(?P<name>\d\d(RR?|L?|C)?))\s+")
COLOR_RE = re.compile(
    r"""^(BLACK)?(BLU|GRN|WHT|RED)\+?
                        (/?(BLACK)?(BLU|GRN|WHT|RED)\+?)*\s*""",
    re.VERBOSE,
)
RUNWAYSTATE_RE = re.compile(
    r"""((?P<name>\d\d) | R(?P<namenew>\d\d)(RR?|LL?|C)?/?)
        ((?P<special> SNOCLO|CLRD(\d\d|//)) |
        (?P<deposit>(\d|/))
        (?P<extent>(\d|/))
        (?P<depth>(\d\d|//))
        (?P<friction>(\d\d|//)))\s+""",
    re.VERBOSE,
)
TREND_RE = re.compile(r"^(?P<trend>TEMPO|BECMG|FCST|NOSIG)\s+")

TRENDTIME_RE = re.compile(r"(?P<when>(FM|TL|AT))(?P<hour>\d\d)(?P<min>\d\d)\s+")

REMARK_RE = re.compile(r"^(RMKS?|NOSPECI|NOSIG)\s+")
UNPARSED_RE = re.compile(r"(?P<group>\S+)\s+")

#m = TYPE_RE.match(message)
#print(message[m.start():m.end()])
    
patterns = [
    #STATION_RE, #Station ID
    #TIME_RE, # Time
    TEMP_RE, # temperature ["temp"]
    TEMP_RE, # dewpt ["dewpt"]
    WIND_RE, # wind
    VISIBILITY_RE, # visibility
    PRESS_RE, # pressure
    #SKY_RE, # sky
    UNPARSED_RE # Remark
]

def sanitize(code):
    """Some string prep to improve parsing fidelity."""
    # Remove extraneous whitespace, any trailing =, then add trailing
    # whitespace as regex matches need that.
    return "%s " % (code.strip().rstrip("="),)

In [65]:
TestMessage = 'SPECI RCSS 020237Z 10015KT 9999 -RA FEW008 BKN018 BKN040 17/16 Q1027 WS R10 NOSIG RMK A3035(Q)=   '
#s_Message = sanitize(TestMessage)
decode(TestMessage)
#print(ConvertToCSV(TestMessage))
#m = REMARK_RE.match(s_Message)
#print(s_Message[m.start():m.end()])
#s_Message = s_Message[m.end():]
#m = COR_RE.match(s_Message)
#print(s_Message[m.start():m.end()])

station: RCSS
type: special report, cycle 2 (automatic report)
time: Thu Apr  2 02:37:00 2020
temperature: 17.0 C
dew point: 16.0 C
wind: E at 15 knots
visibility: greater than 10000 meters
pressure: 1027.0 mb
weather: light rain
sky: a few clouds at 800 feet
     broken clouds at 1800 feet
     broken clouds at 4000 feet
- A3035(Q)=
METAR: SPECI RCSS 020237Z 10015KT 9999 -RA FEW008 BKN018 BKN040 17/16 Q1027 WS R10 NOSIG RMK A3035(Q)=   


/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 020237Z 10015KT 9999 -RA FEW008 BKN018 BKN040 17/16 Q1027 WS R10 NOSIG RMK A3035(Q)=   '
  warnings.warn(message, RuntimeWarning)


In [6]:
#File Splite
source_file = open("Data.csv", "r")
METAR_data_file = open("Metar.csv","w")
SPECI_data_file = open("Speci.csv","w")

for data in source_file:
    if data[7:12] == 'METAR':
        METAR_data_file.write(data)
    else : # SPECI
        SPECI_data_file.write(data)
    
source_file.close()
METAR_data_file.close()
SPECI_data_file.close()

In [66]:
data_file = open("Data.csv", "r")
data_file_output = open("DataOutput.csv","w")

header = [ 'RawMessage', 'Type', 'StationID',
            'Time','temperature',
          'dewpt', 'wind', 'visibility',
          'pressure','sky', 'Remark']

data_file_output.write('\t'.join(header) + '\n')

for data in data_file:
    if data[7:12] == 'METAR':
        data_file_output.write(data.replace('\n','') + '\t')
        data_file_output.write('METAR' + '\t')
        try:
            data_file_output.write(ConvertToCSV(data[7:]) + '\n')
        except :
            data_file_output.write('\n')
    else : # SPECI
        data_file_output.write(data.replace('\n','') + '\t')
        data_file_output.write('SPECI' + '\t')
        try:
            data_file_output.write(ConvertToCSV(data[7:]) + '\n')
        except :
            data_file_output.write('\n')
    
data_file.close()
data_file_output.close()

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 020237Z 10015KT 9999 -RA FEW008 BKN018 BKN040 17/16 Q1027 WS R10 NOSIG RMK A3035(Q)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 020300Z 09012KT 9999 -RA FEW008 BKN020 BKN040 18/16 Q1027 WS R10 NOSIG RMK A3034(R)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 070448Z 08012KT 050V110 9999 FEW016 SCT035 BKN080 24/19 Q1019 WS R10 NOSIG RMK A3010(A)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 070500Z 08012KT 050V110 9999 FEW020 BKN050 BKN120 24/19

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 130506Z 08014KT 9999 FEW025 SCT050 BKN090 26/15 Q1019 WS R10 NOSIG RMK A3009(K)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 130530Z 08015KT 9999 FEW025 SCT045 BKN220 26/15 Q1018 WS R10 NOSIG RMK A3008(L)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 130600Z 08015G25KT 9999 FEW020 BKN045 BKN200 25/14 Q1018 WS R10 NOSIG RMK A3007(M)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 131131Z 11011KT 080V140 9999 FEW025 SCT040 20/14 Q1020 WS R10 NOSIG 

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'SPECI RCSS 070705Z 27015KT 9000 FEW020 31/22 Q1010 WS R28 NOSIG RMK 26014KT/R10 A2983(G)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 070730Z 27015KT 9999 FEW020 SCT050 BKN160 30/21 Q1009 WS R28 NOSIG RMK 27015KT/R10 A2982(H)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 070800Z 32010KT 290V350 9000 FEW020 SCT040 BKN100 28/23 Q1010 WS R28 NOSIG RMK 31006KT/R10 A2983(I)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 070830Z 32009KT 290V360 9000 FEW020 BKN05

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 250730Z 27013KT 9999 FEW025 35/22 Q1005 WS R28 NOSIG RMK 26009KT/R10 A2970(R)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 250800Z 27011KT 9999 FEW025 BKN150 34/21 Q1006 WS R28 NOSIG RMK 26006KT/R10 A2971(T)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 250812Z 27007KT 240V350 9999 FEW025 BKN150 34/21 Q1006 WS R10 NOSIG RMK A2972(U)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 250830Z 10010KT 9999 FEW025 BKN150 30/24 Q1006 WS R10 NOSIG RMK A29

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'SPECI RCSS 160511Z 28018KT 9999 FEW018 SCT035 BKN070 32/25 Q1005 WS R28 NOSIG RMK 28014KT/R10 A2968(P)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R28' while processing 'METAR RCSS 160530Z 28014KT 9999 VCSH FEW012 SCT028 BKN060 32/26 Q1005 WS R28 NOSIG RMK 28010KT/R10 A2968(Q)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 171155Z 09006KT 9999 FEW012 BKN060 26/24 Q1004 WS R10 NOSIG RMK A2965(B)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 171200Z 09006KT 9999 FEW012 BKN060 26/24 Q1004 WS

...

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 232118Z 08018KT 9999 VCSH FEW012 BKN030 BKN050 29/25 Q1002 WS R10 NOSIG RMK A2960(D)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 232201Z 08017G28KT 9999 -SHRA FEW010 BKN025 BKN050 28/25 Q1003 WS R10 NOSIG RMK A2962(G)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 232230Z 08016KT 060V120 9999 -SHRA FEW010 BKN025 BKN050 28/25 Q1003 WS R10 NOSIG RMK A2963(H)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 232300Z 08017G28KT 9999 -SHRA FEW010 SCT025

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 070618Z VRB06KT 9999 VCSH FEW018 FEW020TCU SCT040 BKN200 33/26 Q1002 WS R10 NOSIG RMK TCU E AND S A2960(R)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 070628Z 12013KT 9999 -SHRA FEW018 FEW020TCU BKN035 BKN055 31/24 Q1002 WS R10 NOSIG RMK TCU E A2960(S)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 070630Z 12013KT 9999 -SHRA FEW018 FEW020TCU BKN035 BKN055 31/24 Q1002 RETS WS R10 NOSIG RMK TCU E A2960(T)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR 

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 290502Z 08014KT 030V100 9999 -SHRA FEW012 SCT025 BKN050 31/26 Q1011 WS R10 NOSIG RMK A2986(V)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 290519Z 07011KT 040V100 9999 FEW012 SCT030 BKN060 31/26 Q1011 WS R10 NOSIG RMK A2986(W)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 290530Z 07013G23KT 9999 FEW012 SCT030 BKN060 31/26 Q1010 WS R10 NOSIG RMK A2985(X)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 290600Z 07014KT 040V100 9999 FEW008 SCT018 BKN

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 310802Z 08012KT 040V100 9999 -RA FEW007 SCT028 BKN050 24/22 Q1015 WS R10 NOSIG RMK A3000(H)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 310830Z 08014KT 9000 -RA FEW007 BKN020 BKN038 24/22 Q1016 WS R10 NOSIG RMK A3001(I)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 310848Z 08013KT 5000 -RA FEW007 BKN018 BKN035 24/22 Q1016 WS R10 NOSIG RMK A3001(J)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 010318Z 08015KT 9999 FEW012 BKN028 BKN060 26/20 Q10

/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 270354Z 09019KT 9999 VCSH FEW008 SCT020 BKN100 23/18 Q1020 WS R10 NOSIG RMK A3014(M)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 270400Z 09018KT 9999 VCSH FEW012 SCT020 BKN100 23/18 Q1020 WS R10 NOSIG RMK A3014(N)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'SPECI RCSS 270418Z 08015KT 9999 FEW014 SCT025 BKN100 23/18 Q1020 WS R10 NOSIG RMK A3012(P)=   
'
  warnings.warn(message, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/metar/Metar.py:437: RuntimeWarning: Unparsed groups in body 'WS R10' while processing 'METAR RCSS 270430Z 08016KT 9999 FEW014 SCT025 BKN100 23/18 Q1020 WS R10 